In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import random
import math

# 生成随机城市坐标
def generate_cities(num_cities, width=800, height=600):
    return [(random.randint(50, width-50), random.randint(50, height-50)) for _ in range(num_cities)]

# 计算路径总长度
def calculate_distance(cities, order):
    total = 0
    for i in range(len(order)):
        city_a = cities[order[i]]
        city_b = cities[order[(i+1)%len(order)]]
        total += math.sqrt((city_a[0]-city_b[0])**2 + (city_a[1]-city_b[1])**2)
    return total

# 生成邻域解（交换两个城市）
def get_neighbor(order):
    new_order = order.copy()
    a, b = random.sample(range(len(order)), 2)
    new_order[a], new_order[b] = new_order[b], new_order[a]
    return new_order

# 模拟退火算法
def simulated_annealing(cities, initial_temp=10000, cooling_rate=0.003, iterations=1000):
    current_order = list(range(len(cities)))
    random.shuffle(current_order)
    current_distance = calculate_distance(cities, current_order)
    
    best_order = current_order.copy()
    best_distance = current_distance
    
    temp = initial_temp
    
    # 存储历史记录用于动画
    history = []
    
    for i in range(iterations):
        neighbor_order = get_neighbor(current_order)
        neighbor_distance = calculate_distance(cities, neighbor_order)
        
        # 决定是否接受新解
        if neighbor_distance < current_distance or \
           random.random() < math.exp((current_distance - neighbor_distance) / temp):
            current_order = neighbor_order
            current_distance = neighbor_distance
            
            if current_distance < best_distance:
                best_order = current_order.copy()
                best_distance = current_distance
        
        # 存储当前状态用于动画
        if i % 10 == 0 or i == iterations-1:  # 每10次记录一次，确保动画不会太大
            history.append((current_order.copy(), current_distance, best_order.copy(), best_distance, temp))
        
        # 降低温度
        temp *= 1 - cooling_rate
    
    return best_order, best_distance, history

# 创建动画
def create_animation(cities, history, interval=100):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    plt.close()  # 防止在notebook中显示静态图
    
    # 初始化图形
    def init():
        ax1.clear()
        ax2.clear()
        
        ax1.set_title('Current Solution')
        ax1.set_xlim(0, 800)
        ax1.set_ylim(0, 600)
        
        ax2.set_title('Best Solution So Far')
        ax2.set_xlim(0, 800)
        ax2.set_ylim(0, 600)
        
        return fig,
    
    # 更新函数
    def update(frame):
        current_order, current_distance, best_order, best_distance, temp = frame
        
        ax1.clear()
        ax2.clear()
        
        # 绘制当前解
        ax1.set_title(f'Current Solution\nDistance: {current_distance:.2f}\nTemperature: {temp:.2f}')
        ax1.set_xlim(0, 800)
        ax1.set_ylim(0, 600)
        
        # 绘制城市点
        for i, (x, y) in enumerate(cities):
            ax1.scatter(x, y, c='red')
            ax1.text(x, y, str(i), fontsize=8)
        
        # 绘制路径
        for i in range(len(current_order)):
            city_a = cities[current_order[i]]
            city_b = cities[current_order[(i+1)%len(current_order)]]
            ax1.plot([city_a[0], city_b[0]], [city_a[1], city_b[1]], 'b-')
        
        # 绘制最优解
        ax2.set_title(f'Best Solution So Far\nDistance: {best_distance:.2f}')
        ax2.set_xlim(0, 800)
        ax2.set_ylim(0, 600)
        
        # 绘制城市点
        for i, (x, y) in enumerate(cities):
            ax2.scatter(x, y, c='red')
            ax2.text(x, y, str(i), fontsize=8)
        
        # 绘制路径
        for i in range(len(best_order)):
            city_a = cities[best_order[i]]
            city_b = cities[best_order[(i+1)%len(best_order)]]
            ax2.plot([city_a[0], city_b[0]], [city_a[1], city_b[1]], 'g-')
        
        return fig,
    
    # 创建动画
    anim = FuncAnimation(fig, update, frames=history, init_func=init,
                         interval=interval, blit=False, repeat=False)
    
    return anim


# 生成城市
num_cities = 15
cities = generate_cities(num_cities)

# 运行模拟退火
best_order, best_distance, history = simulated_annealing(cities)

print(f"Best distance found: {best_distance}")
print("Best order:", best_order)

# 创建并显示动画
anim = create_animation(cities, history, interval=200)
# HTML(anim.to_html5_video())

anim.save('./assets/tsp_sa.gif', writer='pillow', fps=5)

Best distance found: 2443.1540287939492
Best order: [4, 5, 7, 3, 13, 6, 2, 0, 8, 10, 1, 12, 14, 11, 9]
